In [ ]:
import pandas as pd
import os

In [ ]:
%load_ext autoreload
%autoreload 2
from EndEffectorPoseOptimizer import *
from JupyterVisualizer import  *

# 0. 매개변수 및 클래스 초기화

In [ ]:
DEBUG_MODE = True

In [ ]:
optimizer = EndEffectorPoseOptimizer(DEBUG_MODE)

# 1. 데이터 로드

### 1.1. 스캔 데이터 로드

In [ ]:
# optimizer.load_scan_data('./data/PIPE NO.1_fill.ply')
# optimizer.load_scan_data('./data/PIPE NO.2_fill.ply')
optimizer.load_scan_data('./data/PIPE NO.3_fill.ply')

### 1.2. 검사 대상 좌표 로드

In [ ]:
df = pd.read_csv('./data/PIPE NO.3_fill_testpoint.csv')
df

In [ ]:
detection_points = []
inspection_poinsts = []
for idx, row in df.iterrows():
    inspection_point = (float(row['x']), float(row['y']), float(row['z']))
    detection_point = (
        float(row['x'] + row['dx']),
        float(row['y'] + row['dy']),
        float(row['z'] + row['dz'])
    )
    detection_points.append(detection_point)
    inspection_poinsts.append(inspection_point)

In [ ]:
pl = visualize_pointclouds_simply(optimizer._scan_data, bg_color='white')
radius = 0.005
for i in range(len(detection_points)):
    if detection_points[i] == inspection_poinsts[i]:
        add_sphere(pl, inspection_poinsts[i], radius=radius, color='green')
    else:
        add_sphere(pl, inspection_poinsts[i], radius=radius, color='red')
        add_sphere(pl, detection_points[i], radius=radius, color='blue')

    pl.add_point_labels([inspection_poinsts[i]], [str(i+1)], font_size=20, point_color='black', text_color='black', always_visible=True)

    

### 1.2. 엔드이펙터 로드

In [ ]:
optimizer.load_DDA_from_urdf('./data/robot_models v1.5/robots/rb10_1300e_DDA.urdf')
dda_mesh = optimizer._EndEffectorPoseOptimizer__dda_mesh # type: ignore
optimizer.load_RT_from_urdf('./data/robot_models v1.5/robots/rb10_1300e_RT.urdf')
rt_mesh = optimizer._EndEffectorPoseOptimizer__rt_mesh # type: ignore

### DDA 가시화

In [ ]:
pl = visualize_mesh(dda_mesh, color='blue', bg_color='white') # type: ignore
add_coordinate_frame(pl, length=0.1, size=0.02)

In [ ]:
pl=visualize_mesh(rt_mesh, color='blue', bg_color='white') # type: ignore
add_coordinate_frame(pl, length=0.1, size=0.02)

# 2. 검사 대상 지점 선택

### 2.1. 탐지모드 좌표 선택
- 인덱스 입력

In [ ]:
INSPECTION_POINT_INDEX = 12
INSPECTION_POINT_INDEX -= 1

In [ ]:
# 관심 대상 주변 점군만 추출
center = np.array(detection_points[INSPECTION_POINT_INDEX])
min_b = center - 0.3
max_b = center + 0.3
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_b, max_bound=max_b) # type: ignore
cropped_scan_data = optimizer._scan_data.crop(bbox)

In [ ]:
pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')
pl.add_point_labels([inspection_poinsts[INSPECTION_POINT_INDEX]], [str(INSPECTION_POINT_INDEX+1)], font_size=20, point_color='black', text_color='black', always_visible=True)
if detection_points[INSPECTION_POINT_INDEX] == inspection_poinsts[INSPECTION_POINT_INDEX]:
    add_sphere(pl, inspection_poinsts[INSPECTION_POINT_INDEX], radius=radius, color='green')
else:
    add_sphere(pl, inspection_poinsts[INSPECTION_POINT_INDEX], radius=radius, color='red')
    add_sphere(pl, detection_points[INSPECTION_POINT_INDEX], radius=radius, color='blue')


# 3. 배관 프로파일 계산

In [ ]:
optimizer.calculate_pipe_profile(detection_points[INSPECTION_POINT_INDEX]) # type: ignore

center = optimizer._EndEffectorPoseOptimizer__pipe_center # type: ignore
direction = optimizer._EndEffectorPoseOptimizer__pipe_direction # type: ignore
radius = optimizer._EndEffectorPoseOptimizer__pipe_radius # type: ignore

pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')
pl.add_arrows(center, direction, 0.1, color="red") # type: ignore
add_cylinder(
    pl, # type: ignore
    center,
    direction,
    radius + 0.003,
    0.05,
    "blue",
    0.3,
)

pl.set_focus(detection_points[INSPECTION_POINT_INDEX]) #type: ignore
pl.camera.zoom(2) # type: ignore


In [ ]:
if DEBUG_MODE:
    sampleing_box = optimizer.debuging_info.get("sampling_box")
    selected_points = optimizer.debuging_info.get("selected_points")
    normal_m = optimizer.debuging_info.get("normal_m")
    points_in_cylinder = optimizer.debuging_info.get("points_in_cylinder")
    estimated_center = optimizer.debuging_info.get("estimated_center")
    estimated_radius = optimizer.debuging_info.get("estimated_radius")

    pl = Plotter(shape=(2, 2))

    # 법선 계산을 위해 추출할 영역 확인
    if sampleing_box is not None:
        pl.subplot(0, 0)
        visualize_pointclouds_simply(cropped_scan_data, plotter=pl, bg_color='white')
        add_box(pl, sampleing_box[0], sampleing_box[1], color="red", opacity=0.3)


    # 법선 계산을 위해 추출한 점 확인
    if selected_points is not None:
        pl.subplot(0, 1)
        visualize_pointclouds_simply(cropped_scan_data, plotter=pl, bg_color='white')
        add_points(pl, selected_points, 3)

        # compute point 1 unit away from detection_points[INDEX] along normal_m
        camera_position = np.array(detection_points[INSPECTION_POINT_INDEX]) + (normal_m / np.linalg.norm(normal_m)) * 5 # type: ignore

        # 카메라 설정을 올바른 순서로 적용
        pl.set_position(camera_position)
        pl.set_viewup((1, 0, 0)) # type: ignore
        pl.set_focus(detection_points[INSPECTION_POINT_INDEX])

        # pl.camera.zoom(15)

    # 직경 계산을 위체 추출한 점 확인
    if points_in_cylinder is not None:
        pl.subplot(1, 0)

        visualize_pointclouds_simply(cropped_scan_data, plotter=pl, bg_color='white')
        points_pc = PointCloud()
        points_pc.points = Vector3dVector(points_in_cylinder)
        add_points(pl, points_pc, 3)
        pl.set_focus(detection_points[INSPECTION_POINT_INDEX])
        # pl.camera.zoom(2)

    # 구체 샘플링 범위 확인
    if normal_m is not None and estimated_center is not None and estimated_radius is not None:
        pl.subplot(1, 1)
        visualize_pointclouds_simply(cropped_scan_data, plotter=pl, bg_color='white')
        add_sphere(pl, estimated_center, radius=estimated_radius + 0.003, color="red", opacity=0.3)
        pl.set_focus(estimated_center)
        # pl.camera.zoom(20)

    pl.show()

In [ ]:
selected_points

# 4. 엔드이펙터 위치 결정(탐색)

In [ ]:
dda_tcp_candidates_filtered_json, dda_tcp_pose_candidates_filtered, dda_tcp_pose_candidates = (
    optimizer.calculate_DDA_pose_for_detecting_welding_point(
        detection_points[INSPECTION_POINT_INDEX],
        8,
        0.2,
    )
)

### 위치 후보 가시화(화살표만)

In [ ]:
arrows= calculate_arrows_from_pose_candidates(dda_tcp_pose_candidates)
pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')
_=pl.add_arrows(arrows[..., 0:3], arrows[..., 3:6], 0.1, color="red") #type: ignore

### 위치 후보 가시화(모델+방향)

In [ ]:
arrows= calculate_arrows_from_pose_candidates(dda_tcp_pose_candidates)
pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')
_=pl.add_arrows(arrows[..., 0:3], arrows[..., 3:6], 0.1, color="red") #type: ignore

for dda_tcp_pose in dda_tcp_pose_candidates:
    rot = R.from_euler("xyz", dda_tcp_pose[3:]).as_matrix()
    dda_tcp_pose_T = np.eye(4)
    dda_tcp_pose_T[:3, :3] = rot
    dda_tcp_pose_T[:3,  3] = dda_tcp_pose[:3]

    tcp_to_dda = optimizer._EndEffectorPoseOptimizer__dda_invers_transform_mat # type: ignore
    dda_pose_T = dda_tcp_pose_T @ tcp_to_dda

    add_mesh(pl, dda_mesh, dda_pose_T, color="blue") # type: ignore

pl.set_focus(detection_points[INSPECTION_POINT_INDEX]) #type: ignore

### 위치 후보 가시화(배관과 충돌하는 자세 제외)

In [ ]:
arrows= calculate_arrows_from_pose_candidates(dda_tcp_pose_candidates_filtered)
pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')
_=pl.add_arrows(arrows[..., 0:3], arrows[..., 3:6], 0.1, color="red") #type: ignore

for dda_tcp_pose in dda_tcp_pose_candidates_filtered:
    rot = R.from_euler("xyz", dda_tcp_pose[3:]).as_matrix()
    dda_tcp_pose_T = np.eye(4)
    dda_tcp_pose_T[:3, :3] = rot
    dda_tcp_pose_T[:3,  3] = dda_tcp_pose[:3]

    tcp_to_dda = optimizer._EndEffectorPoseOptimizer__dda_invers_transform_mat # type: ignore
    dda_pose_T = dda_tcp_pose_T @ tcp_to_dda

    add_mesh(pl, dda_mesh, dda_pose_T, color="blue") # type: ignore

pl.set_focus(detection_points[INSPECTION_POINT_INDEX]) #type: ignore

# 5. 엔드이펙터 위치 결정(검사)

In [ ]:
json_str, dda_rt_pairs= optimizer.calculate_DDA_RT_pose_for_taking_xray(
    inspection_poinsts[INSPECTION_POINT_INDEX],
    8,
    0.01,
    0.3,
    10,
)

In [ ]:
dda_rt_pairs

### DDA 자세 후보 가시화(충돌 제외)

In [ ]:
# Visualize point cloud and DDA with applied pose
# Visualize point cloud and multiple DDA meshes from dda_poses
pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')

for pair in dda_rt_pairs:
    pose = pair["0"]['DDA']
    rot = R.from_euler("xyz", pose[3:]).as_matrix()
    dda_tcp_pose_T = np.eye(4)
    dda_tcp_pose_T[:3, :3] = rot
    dda_tcp_pose_T[:3, 3] = pose[:3]

    tcp_to_dda = optimizer._EndEffectorPoseOptimizer__dda_invers_transform_mat  # type: ignore
    dda_pose_T = dda_tcp_pose_T @ tcp_to_dda

    add_mesh(pl, dda_mesh, dda_pose_T, color='blue')  # type: ignore

pl.set_focus(detection_points[INSPECTION_POINT_INDEX])

In [ ]:
def vis_inspection_points(pl, inspection_points, detection_points, radius=0.005):
    for i in range(len(detection_points)):
        if detection_points[i] == inspection_points[i]:
            add_sphere(pl, inspection_points[i], radius=radius, color='green')
        else:
            add_sphere(pl, inspection_points[i], radius=radius, color='red')
            add_sphere(pl, detection_points[i], radius=radius, color='blue')

        pl.add_point_labels([inspection_points[i]], [str(i + 1)], font_size=20, point_color='black', text_color='black', always_visible=True)

def vis_tcp_x_axis(pl:Plotter, dda_rt_pair:dict[str, np.ndarray], mag=0.15, color:str|tuple='red'):
    dda_pose = dda_rt_pair['DDA']
    rt_pose = dda_rt_pair['RT1'] if 'RT1' in dda_rt_pair else dda_rt_pair['RT2']

    arrows = calculate_arrows_from_pose_candidates(np.vstack([dda_pose, rt_pose]))
    _ = pl.add_arrows(arrows[..., 0:3], arrows[..., 3:6], mag, color=color)

def vis_end_effector_mesh(pl:Plotter, dda_rt_pair:dict[str, np.ndarray], optimizer:EndEffectorPoseOptimizer, color:str|tuple='gray'):
    dda_pose = dda_rt_pair['DDA']
    rot = R.from_euler("xyz", dda_pose[3:]).as_matrix()
    dda_tcp_pose_T = np.eye(4)
    dda_tcp_pose_T[:3, :3] = rot
    dda_tcp_pose_T[:3,  3] = dda_pose[:3]

    tcp_to_dda = optimizer._EndEffectorPoseOptimizer__dda_invers_transform_mat # type: ignore
    dda_pose_T = dda_tcp_pose_T @ tcp_to_dda

    add_mesh(pl, optimizer._EndEffectorPoseOptimizer__dda_mesh, dda_pose_T, color=color) # type: ignore

    rt_pose = dda_rt_pair['RT1'] if 'RT1' in dda_rt_pair else dda_rt_pair['RT2']
    rot = R.from_euler("xyz", rt_pose[3:]).as_matrix()
    rt_tcp_pose_T = np.eye(4)
    rt_tcp_pose_T[:3, :3] = rot
    rt_tcp_pose_T[:3,  3] = rt_pose[:3]

    tcp_to_rt = optimizer._EndEffectorPoseOptimizer__rt_invers_transform_mat # type: ignore
    rt_pose_T = rt_tcp_pose_T @ tcp_to_rt

    add_mesh(pl, optimizer._EndEffectorPoseOptimizer__rt_mesh, rt_pose_T, color=color) # type: ignore

In [ ]:
PAIR_IDX = 0
GRID_ROW = 3
GRID_COL = 4

pl = Plotter(shape=(GRID_ROW, GRID_COL), window_size=[1800, 1100])
pl.show_axes_all()

# 스캔 데이터 및 검사 지점 가시화=======================================================================
for row in range(GRID_ROW):
    for col in range(GRID_COL):
        pl.subplot(row, col)
        visualize_pointclouds_simply(cropped_scan_data, plotter=pl, bg_color='white')
        pl.add_point_labels([inspection_poinsts[INSPECTION_POINT_INDEX]], [str(INSPECTION_POINT_INDEX+1)], font_size=20, point_color='black', text_color='black', always_visible=True)

# 엔드 이펙터 기본 위치 가시화==========================================================================
for col in range(GRID_COL):
    color = (255, 150, 150)
    # 개별 가시화-----------------------------------------------------------------
    pl.subplot(0, col)
    # vis_end_effector_mesh(pl, dda_rt_pairs[PAIR_IDX]['0'], optimizer, color='gray')
    vis_end_effector_mesh(pl, dda_rt_pairs[PAIR_IDX]['0'], optimizer, color=color)
    vis_tcp_x_axis(pl, dda_rt_pairs[PAIR_IDX]['0'], mag=0.15)

    # 통합 가시화-----------------------------------------------------------------
    pl.subplot(2, col)
    vis_end_effector_mesh(pl, dda_rt_pairs[PAIR_IDX]['0'], optimizer, color=color)
    vis_tcp_x_axis(pl, dda_rt_pairs[PAIR_IDX]['0'], mag=0.15)

# 엔드 이펙터 90도 회전 위치 가시화=====================================================================
for col in range(GRID_COL):
    color = (150, 150, 255)
    # 개별 가시화-----------------------------------------------------------------
    pl.subplot(1, col)
    vis_end_effector_mesh(pl, dda_rt_pairs[PAIR_IDX]['90'], optimizer, color=color)
    vis_tcp_x_axis(pl, dda_rt_pairs[PAIR_IDX]['90'], mag=0.15, color='blue')

    # 통합 가시화-----------------------------------------------------------------
    pl.subplot(2, col)
    vis_end_effector_mesh(pl, dda_rt_pairs[PAIR_IDX]['90'], optimizer, color=color)
    vis_tcp_x_axis(pl, dda_rt_pairs[PAIR_IDX]['90'], mag=0.15, color='blue')

# 카메라 시점 설정====================================================================================
for col in range(GRID_COL):
    for row in range(GRID_ROW):
        pl.subplot(row, col)

        if col == 0:
            pl.view_isometric() # type: ignore
        elif col == 1:
            pl.view_yz() # type: ignore
        elif col == 2:
            pl.view_xz() # type: ignore
        elif col == 3:
            pl.view_xy() # type: ignore

pl.show()

os.makedirs('./한화오션 지정 포인트 실험 결과', exist_ok=True)
_ = pl.screenshot(f'./한화오션 지정 포인트 실험 결과/{INSPECTION_POINT_INDEX+1}.jpg')

In [ ]:
# PAIR_IDX = 0

# # 점군 가시화
# pl = visualize_pointclouds_simply(cropped_scan_data, bg_color='white')

# # 검사 위치 가시화
# radius = 0.005
# if detection_points[INSPECTION_POINT_INDEX] == inspection_poinsts[INSPECTION_POINT_INDEX]:
#     add_sphere(pl, inspection_poinsts[INSPECTION_POINT_INDEX], radius=radius, color='green')
# else:
#     add_sphere(pl, inspection_poinsts[INSPECTION_POINT_INDEX], radius=radius, color='red')
#     add_sphere(pl, detection_points[INSPECTION_POINT_INDEX], radius=radius, color='blue')

# pl.add_point_labels([inspection_poinsts[INSPECTION_POINT_INDEX]], [str(INSPECTION_POINT_INDEX+1)], font_size=20, point_color='black', text_color='black', always_visible=True)

# # 엔드 이펙터 x축 가시화
# arrows = calculate_arrows_from_pose_candidates(np.vstack([dda_poses[PAIR_IDX], rt_poses[PAIR_IDX]]))
# _=pl.add_arrows(arrows[..., 0:3], arrows[..., 3:6], 0.15, color="red")

# # DDA 형상 가시화
# rot = R.from_euler("xyz", dda_poses[PAIR_IDX][3:]).as_matrix()
# dda_tcp_pose_T = np.eye(4)
# dda_tcp_pose_T[:3, :3] = rot
# dda_tcp_pose_T[:3, 3] = dda_poses[PAIR_IDX][:3]
# tcp_to_dda = optimizer._EndEffectorPoseOptimizer__dda_invers_transform_mat  # type: ignore
# dda_pose_T = dda_tcp_pose_T @ tcp_to_dda
# add_mesh(pl, dda_mesh, dda_pose_T, color='gray')  # type: ignore

# # RT 형상 가시화
# rot = R.from_euler("xyz", rt_poses[PAIR_IDX][3:]).as_matrix()
# rt_tcp_pose_T = np.eye(4)
# rt_tcp_pose_T[:3, :3] = rot
# rt_tcp_pose_T[:3, 3] = rt_poses[PAIR_IDX][:3]
# tcp_to_rt = optimizer._EndEffectorPoseOptimizer__rt_invers_transform_mat  # type: ignore
# rt_pose_T = rt_tcp_pose_T @ tcp_to_rt
# add_mesh(pl, rt_mesh, rt_pose_T, color='gray')  # type: ignore

# # 카메라 설정
# pl.set_focus(detection_points[INSPECTION_POINT_INDEX])